In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import csv
import cv2
import json
import scipy
import random

from pathlib import Path
from collections import defaultdict
from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import torch.nn.functional as F
import torch.autograd as autograd
from torchvision import transforms

In [ ]:
PATH = Path('../data/')

In [ ]:
list(PATH.iterdir())

In [ ]:
train_labels = pd.read_csv(str(PATH/'traininglabels.csv'))

In [ ]:
train_labels.head(5)

In [ ]:
np.sum(train_labels['has_oilpalm'])

In [ ]:
len(train_labels)

In [ ]:
np.sum(train_labels['has_oilpalm'])/len(train_labels)

# Check train images

In [ ]:
def load_image(img_path):
    img = cv2.imread(str(img_path)).astype(np.float32)/255
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img_rgb

def show_image(paths, nrow=2, ncol=3):
    n = nrow * ncol
    fig, axes = plt.subplots(nrow, ncol, figsize=(12, 8))
    
    for i, ax in enumerate(axes.flat):
        img = load_image(paths[i][0])
        ax.imshow(img)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        if paths[i][1] == 0:
            ax.set_title(f'No oil palm trees, label score is {paths[i][2]}')
        else:
            ax.set_title(f'Has oil palm trees, label score is {paths[i][2]}')
    
    plt.tight_layout()

In [ ]:
img_names = random.sample(list(train_labels['image_id']), 6)
img_labels = [train_labels.loc[train_labels['image_id'] == p, 'has_oilpalm'].values[0] for p in img_names]
label_score = [train_labels.loc[train_labels['image_id'] == p, 'score'].values[0] for p in img_names]

img_paths = [(str(PATH/f"train_images/{p}"), l, s) for p, l, s in zip(img_names, img_labels, label_score)]
img_paths

In [ ]:
show_image(img_paths)

# Data Loader

In [ ]:
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [ ]:
# all images are 256x256
img_size = 128
batch_size = 64

In [ ]:
def normalize(im):
    """Normalizes images with Imagenet stats."""
    imagenet_stats = np.array([[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]])
    return (im - imagenet_stats[0])/imagenet_stats[1]

def denormalize(im):
    """Denormalizes images."""
    imagenet_stats = np.array([[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]])
    return (im * imagenet_stats[1]) + imagenet_stats[0]

## Data augmentation

In [ ]:
def rotate_cv(im, deg, mode=cv2.BORDER_REFLECT, interpolation=cv2.INTER_AREA):
    """ Rotates an image by deg degrees"""
    r,c,*_ = im.shape
    M = cv2.getRotationMatrix2D((c/2,r/2),deg,1)
    return cv2.warpAffine(im,M,(c,r), borderMode=mode, flags=cv2.WARP_FILL_OUTLIERS+interpolation)

## Dataset

In [ ]:
class oilPalmDataset(Dataset):
    def __init__(self, dataset, size, transform=False, test=False):
        self.dataset = dataset[:, 0]
        self.test = test
        self.transforms = transform
        self.sz = size
        if not self.test:
            self.y = dataset[:, 1]
    
    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, i):
        path = self.dataset[i]
        X = load_image(path)
        if not self.test:
            X = cv2.resize(X, (self.sz, self.sz))
        if self.transforms:
            random_degree = (np.random.random()-.50)*10 # +5/-5 degrees
            X = rotate_cv(X, random_degree)
            if np.random.random() > 0.5: # .5 prob random flip
                X = np.fliplr(X).copy()
        X = normalize(X)
        X = X.transpose(2, 0, 1)
        if not self.test:
            y = int(self.y[i])
            return X, y
        else:
            return X, str(path).split('/')[-1]

In [ ]:
X = list(train_labels.loc[:, 'image_id'])
X = [str(PATH/f"train_images/{p}") for p in X]
y = list(train_labels.loc[:, 'has_oilpalm'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

Since the training set is unbalanced, let's check if the train/valid dataset is stratified

In [ ]:
np.sum(y_train)/(len(y)*.8)

In [ ]:
np.sum(y_val)/(len(y)*.2)

## Make data loader

In [ ]:
train = np.concatenate([np.array(X_train).reshape(-1, 1), np.array(y_train).reshape(-1, 1)], axis=1)
valid = np.concatenate([np.array(X_val).reshape(-1, 1), np.array(y_val).reshape(-1, 1)], axis=1)

In [ ]:
train = oilPalmDataset(train, size=img_size, transform=True)
valid = oilPalmDataset(valid, size=img_size)

In [ ]:
train_dl = DataLoader(train, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid, batch_size=batch_size)

In [ ]:
X, y = next(iter(train_dl))
X.shape, y.shape

### Modelling

In [ ]:
class Net(nn.Module):
    def __init__(self, num_classes=2):
        super(Net, self).__init__()
        resnet = models.resnet50(pretrained=True)
        for param in resnet.parameters():
            param.requires_grad = False
        layers = list(resnet.children())[:-2]
        self.top_model = nn.Sequential(*layers).cuda()
        self.out = 2048
        self.bn1 = nn.BatchNorm1d(self.out)
        self.bn2 = nn.BatchNorm1d(2048)
        self.fc1 = nn.Linear(self.out, 2048)
        self.fc2 = nn.Linear(2048, num_classes)
    
    def forward(self, x):
        x = F.relu(self.top_model(x))
        x = nn.AdaptiveAvgPool2d((1,1))(x)
        x = x.view(x.shape[0], -1)
        x = nn.Dropout(0.2)(x)
        x = self.bn1(x)
        x = F.relu(self.fc1(x))
        x = nn.Dropout(0.2)(x)
        x = self.bn2(x)
        x = self.fc2(x)
        return x

In [ ]:
def get_optimizer(model, lr = 0.01, wd = 0.0):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optim = torch.optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim


def LR_range_finder(model, train_dl, lr_low=1e-6, lr_high=1, epochs=2):
    losses = []
    iterations = epochs * len(train_dl)
    delta = (lr_high - lr_low)/iterations
    lrs = [lr_low + i*delta for i in range(iterations)]
    model.train()
    ind = 0
    for i in range(epochs):
        for x, y in train_dl:
            optim = get_optimizer(model, lr=lrs[ind])
            x = x.cuda().float()
            y = y.cuda()
            out = model(x)
            loss = F.cross_entropy(out, y)
            optim.zero_grad()
            loss.backward()
            optim.step()
            losses.append(loss.item())
            ind +=1

    return lrs, losses 


def save_model(m, p): 
    torch.save(m.state_dict(), p)
    
def load_model(m, p): 
    m.load_state_dict(torch.load(p))

In [ ]:
model = Net().cuda()
lrs, losses = LR_range_finder(model, train_dl)

In [ ]:
plt.figure(figsize=(16, 5))
plt.plot(lrs[:100], losses[:100])
plt.show()

In [ ]:
def get_triangular_lr2(lr_low, lr_high, stepesize):
    iterations = 2*stepesize
    iter1 = int(0.35*iterations)
    iter2 = int(0.85*iter1)
    iter3 = iterations - iter1 - iter2
    delta1 = (lr_high - lr_low)/iter1
    delta2 = (lr_high - lr_low)/(iter1 -1)
    lrs1 = [lr_low + i*delta1 for i in range(iter1)]
    lrs2 = [lr_high - i*(delta1) for i in range(0, iter2)]
    delta2 = (lrs2[-1] - lr_low)/(iter3)
    lrs3 = [lrs2[-1] - i*(delta2) for i in range(1, iter3+1)]
    return lrs1+lrs2+lrs3

In [ ]:
def train_triangular_policy(model, train_dl, valid_dl, lr_low=1e-6, lr_high=0.1):
    idx = 0
    epochs = 4
    stepesize = 2*len(train_dl)
    lrs = get_triangular_lr2(lr_low, lr_high, stepesize)
    for i in range(epochs):
        model.train()
        total = 0
        sum_loss = 0
        for i, (x, y) in enumerate(train_dl):
            optim = get_optimizer(model, lr = lrs[idx], wd =0)
            batch = y.shape[0]
            x = x.cuda().float()
            y = y.cuda()
            out = model(x)
            loss = F.cross_entropy(out, y)
            optim.zero_grad()
            loss.backward()
            optim.step()
            idx += 1
            total += batch
            sum_loss += batch*(loss.item())
        print("train loss %.3f" % (sum_loss/total))
        val_metrics(model, valid_dl)
    return sum_loss/total

In [ ]:
def val_metrics(model, valid_dl):
    model.eval()
    total = 0
    sum_loss = 0
    correct = 0 
    for i, (x, y) in enumerate(valid_dl):
        batch = y.shape[0]
        x = x.cuda().float()
        y = y.cuda()
        out = model(x)
#         _, pred = torch.max(out, 1)
#         correct += pred.eq(y).sum().item()
        loss = F.cross_entropy(out, y)
        sum_loss += batch*(loss.item())
        total += batch
    print("val loss %.3f and accuracy score %.3f" % (sum_loss/total, correct/total))

In [ ]:
from datetime import datetime

def training_loop(model, train_dl, valid_dl, steps=3, lr_low=1e-6, lr_high=0.1):
    for i in range(steps):
        start = datetime.now() 
        loss = train_triangular_policy(model, train_dl, valid_dl, lr_low, lr_high)
        end = datetime.now()
        t = 'Time elapsed {}'.format(end - start)
        print("----End of step", t, "\n")

In [ ]:
model = Net().cuda()

In [ ]:
# before training
val_metrics(model, valid_dl)

In [ ]:
training_loop(model, train_dl, valid_dl, steps=1, lr_low=1e-6, lr_high=1e-2)

In [ ]:
p = "./baseline-model-98.pth"
save_model(model, p)

In [ ]:
model.unfreeze(7)
model.unfreeze(6)

# Check validation result

In [ ]:
p = "./baseline-model-98.pth"
model = Net().cuda()
model.load_state_dict(torch.load(p))

In [ ]:
val_out = []

model.eval()
for X, y in valid_dl:
    X = X.float().cuda()
    y_hat = model(X)
    _, pred = torch.max(y_hat, 1)
    val_out.append(pred)

In [ ]:
val_X = []
val_y = []
for X, y in valid_dl:
    val_X.append(X)
    val_y.append(y)

In [ ]:
val_out = [t.cpu().numpy() for t in val_out]
val_out = np.concatenate(val_out).ravel()
val_y = np.concatenate(val_y).ravel()

In [ ]:
val_y.shape, val_out.shape

In [ ]:
np.sum(val_y)

In [ ]:
diff_idx = np.where(~np.equal(val_out, val_y))[0]
diff_idx

In [ ]:
for idx in diff_idx:
    print(f'Truth: {val_y[idx]}, prediction: {val_out[idx]}')

# Test

In [ ]:
TEST_PATH_1 = PATH/'leaderboard_test_data'
TEST_PATH_2 = PATH/'leaderboard_holdout_data'

In [ ]:
X_test_path_1 = np.array(list(TEST_PATH_1.iterdir())).reshape(-1, 1)
X_test_path_2 = np.array(list(TEST_PATH_2.iterdir())).reshape(-1, 1)
X_test_path = np.concatenate([X_test_path_1, X_test_path_2])
X_test_path.shape

In [ ]:
test = oilPalmDataset(X_test_path, size=256, test=True)
test_dl = DataLoader(test, batch_size=batch_size)

In [ ]:
X, paths = next(iter(test))
X.shape, paths

In [ ]:
p = "./baseline-model-98.pth"
model = Net().cuda()
model.load_state_dict(torch.load(p))

In [ ]:
out = []
fpaths = []

model.eval()
for X, path in test_dl:
    X = X.float().cuda()
    y_hat = model(X)
    pred = y_hat[:, 1]
    out.append(pred)
    fpaths.append(path)

In [ ]:
out = [t.detach().cpu().numpy() for t in out]
out = np.concatenate(out).ravel()
fpaths = np.concatenate(fpaths).ravel()

In [ ]:
out.shape

In [ ]:
fpaths.shape

In [ ]:
pred_out = sorted(list(zip(fpaths, out)), key=lambda x: x[0])

with open('test_output_pytorch.csv','w') as f:
    csv_out = csv.writer(f)
    csv_out.writerow(['image_id', 'has_oilpalm'])
    for row in pred_out:
        csv_out.writerow(row)